In [ ]:
import fcm    
import kfcpfskld                 
from utils import reshaping_linear, grounded_imaging, processing, fuzzy_to_crisp, clustering, indexing_final, append_result, tot_dice_score, tot_jaccard_score
from PIL import Image                
import os                          
import pandas as pd                  
import cv2                             
import numpy as np                     
import matplotlib.pyplot as plt        
import scipy.io

In [ ]:

path = r"/Test_Img/GN"
image_orig=cv2.imread(r'/Test_Img/NewImg2.tiff', 0)


#MAKING GROUNDED SETS FOR IMAGE (MODIFY FOR EACH IMAGE IN THE MODULE )
ground_set , image_act = grounded_imaging(image_orig)

img_name=[]          
accuracy_list_final = []
PC_list_final = []
PE_list_final = []
dice_score_list1_final, dice_score_list2_final, dice_score_list3_final, dice_score_list4_final = [], [], [], []
jaccard_score_list1_final, jaccard_score_list2_final, jaccard_score_list3_final, jaccard_score_list4_final = [], [], [], []
FPR_list1_final, FPR_list2_final, FPR_list3_final, FPR_list4_final = [], [], [], []
FNR_list1_final, FNR_list2_final, FNR_list3_final, FNR_list4_final = [], [], [], []
final_images_scores_list = []
optimal_scores_list = []
image_name_list = []
range_list_ = []
dict_final = {}

for imageName in os.listdir(path):
  # if imageName=='NewImg2_0_015.tiff':
    print(imageName)
    inputPath = os.path.join( path , imageName )
    img_name.append(imageName)
    
    #READING IMAGE

    if imageName[-3:]=='mat' :
        image = np.round(scipy.io.loadmat(inputPath)['aa'])
    else :
        image = cv2.imread(inputPath , 0)
    range_list = np.linspace(0.1, 0.6, 6)
    
    dice_score_list_1, dice_score_list_2, dice_score_list_3, dice_score_list_4 = np.zeros(len(range_list)), np.zeros(len(range_list)), np.zeros(len(range_list)), np.zeros(len(range_list))
    jaccard_score_list_1, jaccard_score_list_2, jaccard_score_list_3, jaccard_score_list_4 = np.zeros(len(range_list)), np.zeros(len(range_list)), np.zeros(len(range_list)),np.zeros(len(range_list))
    FPR_list_1, FPR_list_2, FPR_list_3, FPR_list_4 = np.zeros(len(range_list)), np.zeros(len(range_list)), np.zeros(len(range_list)), np.zeros(len(range_list))
    FNR_list_1, FNR_list_2, FNR_list_3, FNR_list_4 = np.zeros(len(range_list)), np.zeros(len(range_list)), np.zeros(len(range_list)), np.zeros(len(range_list))
    accuracy_list = np.zeros(len(range_list))
    PC_list = np.zeros(len(range_list))
    PE_list = np.zeros(len(range_list))

    for index,alpha in enumerate(range_list):
    #USE YOUR DEFINED MODEL HERE
        print(alpha)
        image_name_list.append(imageName)
        range_list_.append( alpha )
        init_u,init_centers = fcm.fit(image,4,10,2,0.01)
        u = kfcpfskld.fit_(image, 4, 2, 0.4, 0.01, 100, init_u , init_centers, alpha,3,150)
    
    #CALCULATE THE DICE_SCORE , JACCARD SCORE , ACCURACY & A CONFUSION MATRIX
        dice_score , jaccard_score , accuracy, FPR, FNR, PC, PE = processing(u , 2 ,image , ground_set , image_act,imageName )
        
        dice_score_list_1[index], dice_score_list_2[index], dice_score_list_3[index], dice_score_list_4[index]= np.array(dice_score[0]), np.array(dice_score[1]), np.array(dice_score[2]), np.array(dice_score[3])
        jaccard_score_list_1[index], jaccard_score_list_2[index], jaccard_score_list_3[index], jaccard_score_list_4[index] = np.array(jaccard_score[0]), np.array(jaccard_score[1]), np.array(jaccard_score[2]), np.array(jaccard_score[3])
        FPR_list_1[index], FPR_list_2[index], FPR_list_3[index], FPR_list_4[index]=np.array(FPR[0]), np.array(FPR[1]), np.array(FPR[2]), np.array(FPR[3])
        FNR_list_1[index], FNR_list_2[index], FNR_list_3[index], FNR_list_4[index]=np.array(FNR[0]), np.array(FNR[1]), np.array(FNR[2]), np.array(FNR[3])
        accuracy_list[index]=np.array(accuracy)
        PC_list[index]=np.array(PC)
        PE_list[index]=np.array(PE)
        
    accuracy_list_final = accuracy_list_final + accuracy_list.tolist()
    PC_list_final = PC_list_final + PC_list.tolist()
    PE_list_final = PE_list_final + PE_list.tolist()
    dice_score_list1_final, dice_score_list2_final, dice_score_list3_final, dice_score_list4_final = dice_score_list1_final + dice_score_list_1.tolist(), dice_score_list2_final + dice_score_list_2.tolist(), dice_score_list3_final + dice_score_list_3.tolist(), dice_score_list4_final + dice_score_list_4.tolist()
    jaccard_score_list1_final, jaccard_score_list2_final, jaccard_score_list3_final, jaccard_score_list4_final = jaccard_score_list1_final + jaccard_score_list_1.tolist(), jaccard_score_list2_final + jaccard_score_list_2.tolist(), jaccard_score_list3_final + jaccard_score_list_3.tolist(), jaccard_score_list4_final + jaccard_score_list_4.tolist()
    FPR_list1_final, FPR_list2_final, FPR_list3_final, FPR_list4_final = FPR_list1_final + FPR_list_1.tolist(), FPR_list2_final + FPR_list_2.tolist(), FPR_list3_final + FPR_list_3.tolist(), FPR_list4_final + FPR_list_4.tolist()
    FNR_list1_final, FNR_list2_final, FNR_list3_final, FNR_list4_final = FNR_list1_final + FNR_list_1.tolist(), FNR_list2_final + FNR_list_2.tolist(), FNR_list3_final + FNR_list_3.tolist(), FNR_list4_final + FNR_list_4.tolist()
    accuracy_array = np.array(accuracy_list)

    max_index = np.argmax(accuracy_array)
    optimal_scores_list.append((imageName , range_list[max_index] , accuracy_array[max_index] , PC_list[max_index] , PE_list[max_index] , dice_score_list_1[max_index] ,dice_score_list_2[max_index] ,dice_score_list_3[max_index] ,dice_score_list_4[max_index] , jaccard_score_list_1[max_index], jaccard_score_list_2[max_index], jaccard_score_list_3[max_index], jaccard_score_list_4[max_index], FPR_list_1[max_index], FPR_list_2[max_index], FPR_list_3[max_index], FPR_list_4[max_index], FNR_list_1[max_index], FNR_list_2[max_index], FNR_list_3[max_index], FNR_list_4[max_index] ))

dict_final = {'image_name': image_name_list  ,'alpha' : range_list_ , 'accuracy' : accuracy_list_final , 'PC' : PC_list_final , 'PE' : PE_list_final , 'avg_dic1' : dice_score_list1_final,'avg_dic2' : dice_score_list2_final,'avg_dic3' : dice_score_list3_final,'avg_dic4' : dice_score_list4_final , 'avg_jac1' : jaccard_score_list1_final,'avg_jac2' : jaccard_score_list2_final ,'avg_jac3' : jaccard_score_list3_final ,'avg_jac4' : jaccard_score_list4_final, 'FPR_1' : FPR_list1_final , 'FPR_2' : FPR_list2_final , 'FPR_3' : FPR_list3_final , 'FPR_4' : FPR_list4_final, 'FNR_1' : FNR_list1_final , 'FNR_2' : FNR_list2_final , 'FNR_3' : FNR_list3_final , 'FNR_4' : FNR_list4_final  }
final_df = pd.DataFrame.from_dict(dict_final, orient='index')
final_df.transpose()
  
optimal_scores_df = pd.DataFrame(optimal_scores_list ,columns =['ImageName', 'alpha', 'accuracy' , 'PC', 'PE', 'dic1', 'dic2', 'dic3', 'dic4' , 'jac1', 'jac2', 'jac3', 'jac4', 'FPR1' , 'FPR2' , 'FPR3' , 'FPR4' , 'FNR1' , 'FNR2' , 'FNR3' , 'FNR4'])



In [ ]:
optimal_scores_df

,ImageName,alpha,accuracy,PC,PE,dic1,dic2,dic3,dic4,jac1,...,jac3,jac4,FPR1,FPR2,FPR3,FPR4,FNR1,FNR2,FNR3,FNR4
0,NewImg2_0_01.tiff,0.4,99.960840,0.350484,0.324521,1.0,0.999974,0.998419,0.999706,1.0,...,0.996843,0.999413,0.0,0.000051,0.000811,0.000431,0.0,0.000000,0.002348,0.000157
1,NewImg2_0_005.tiff,0.3,99.986947,0.351726,0.316714,1.0,1.000000,0.999466,0.999902,1.0,...,0.998932,0.999804,0.0,0.000000,0.000428,0.000117,0.0,0.000000,0.000641,0.000078
2,NewImg2_0_015.tiff,0.3,99.928989,0.351344,0.318767,1.0,0.999770,0.997283,0.999468,1.0,...,0.994581,0.998936,0.0,0.000204,0.003167,0.000454,0.0,0.000255,0.002268,0.000610


In [ ]:
optimal_scores_df

,ImageName,alpha,accuracy,PC,PE,dic1,dic2,dic3,dic4,jac1,...,jac3,jac4,FPR1,FPR2,FPR3,FPR4,FNR1,FNR2,FNR3,FNR4
0,NewImg2_0_15RN.mat,0.3,99.995823,0.351745,0.316620,1.0,1.0,0.999829,0.999969,1.0,...,0.999658,0.999937,0.0,0.0,0.000171,0.000031,0.0,0.0,0.000171,0.000031
1,NewImg2_0_10RN.mat,0.3,99.999478,0.351807,0.316314,1.0,1.0,0.999979,0.999996,1.0,...,0.999957,0.999992,0.0,0.0,0.000043,0.000000,0.0,0.0,0.000000,0.000008
2,NewImg2_0_05RN.mat,0.1,100.000000,0.341281,0.316264,1.0,1.0,1.000000,1.000000,1.0,...,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
